참조 URL : https://github.com/eriklindernoren/PyTorch-YOLOv3

# 필요한 Library 설치

In [ ]:
# PyTorch에서 구현된 YOLOv3 코드 및 데이터를 git을 이용하여 복사
!git clone https://github.com/eriklindernoren/PyTorch-YOLOv3

# darknet, YOLOv3, YOLOv3-tiny의 weight를 bash를 이용하여 다운로드
!bash ./PyTorch-YOLOv3/weights/download_weights.sh

# 구현된 라이브러리 설치
!pip3 install pytorchyolo

# 라이브러리 설치 시 Pytorch 버전과 torchvision 버전이 맞지 않는 이슈 발생
# torchvision을 다시 설치하면 Pytorch와 torchvision 버전이 맞춰짐
!pip3 install torchvision

### 해당 셀 실행 종료 후 런타임 재시작 (Pillow, Matplotlib 최신 버전 사용 시 런타임 재시작 필요) ###

# Inference 및 결과 시각화

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision

import pytorchyolo
from pytorchyolo import detect, models
from pytorchyolo.utils.utils import load_classes

In [ ]:
# YOLO model 불러오기
# yolov3.cfg 파일과 yolov3.weights 파일의 경로를 확인 후 입력
model = models.load_model(
  "./PyTorch-YOLOv3/config/yolov3.cfg", 
  "./yolov3.weights"
  )

model.eval()

# Dataset의 class name 불러오기
# coco.names 파일의 경로를 확인 후 입력
classes = load_classes('./PyTorch-YOLOv3/data/coco.names')

# 시각화를 위해 각 클래스 별 color 설정
bs = np.random.choice( np.arange(256), size=len(classes), replace=False )
gs = np.random.choice( np.arange(256), size=len(classes), replace=False )
rs = np.random.choice( np.arange(256), size=len(classes), replace=False )

cls_color = {}
for class_name, b, g, r in zip(classes, bs, gs, rs):
    cls_color[class_name] = (int(b), int(g), int(r))

In [ ]:
# git으로 받은 폴더 내에 있는 이미지 불러오기
# 입력할 이미지의 경로를 확인 후 입력
img = cv2.imread("./PyTorch-YOLOv3/data/samples/dog.jpg")

# 불러온 이미지 확인
plt.figure( figsize=(10,10) )
plt.imshow( img[..., ::-1] )
plt.show()

In [ ]:
# Detection
# boxes = NMS를 적용한 결과 (x1, y1, x2, y2, confidence, class)
boxes = detect.detect_image(model, img)

# Detection 결과 확인
print('Result Shape :', boxes.shape)
print('Result :\n', boxes)

In [ ]:
# 결과 시각화
plot_image = img.copy()

for (x1, y1, x2, y2, confidence, cls) in boxes:

    class_name = classes[ int(cls) ]

    b, g, r = cls_color[class_name]

    text = f'{class_name} : {confidence:.2f}'
    size, baseline = cv2.getTextSize( text, 4, 0.9, 2 )
    cv2.putText( plot_image, text, (int(x2-size[0]), int(y2+size[1])), 4, 0.9, (b, g, r), 2 )

    cv2.rectangle( plot_image, (int(x1), int(y1)), (int(x2), int(y2)), (b, g, r), 2 )

plt.figure( figsize=(10,10) )
plt.imshow( plot_image[..., ::-1] )
plt.show()

# 모델 구조 코드

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pytorchyolo
from pytorchyolo import models
from pytorchyolo.utils.parse_config import parse_model_config

### 모델 구조 확인 ###
model = models.load_model(
  "./PyTorch-YOLOv3/config/yolov3.cfg", 
  "./yolov3.weights"
)

# pytorch에서는 .eval()을 사용하면 train parameter(dropout, batchnorm 등)를 사용하지 않도록 설정됨
# 또한 셀에서 .eval() 사용 시 모델 구조를 출력하는 기능도 포함되어 있음
model.eval()

#### 모델 구현 코드 URL
- https://github.com/eriklindernoren/PyTorch-YOLOv3/blob/master/pytorchyolo/models.py

#### cfg 파일 확인
- https://github.com/eriklindernoren/PyTorch-YOLOv3/blob/master/config/yolov3.cfg

In [ ]:
# cfg 파일을 parsing하여 모델 구조를 생성함
module_defs = parse_model_config("./PyTorch-YOLOv3/config/yolov3.cfg") # cfg 파일을 parsing 하는 함수

for module_def in module_defs:
    print( module_def )

In [ ]:
# 각 parameter의 name과 weight를 확인할 수 있음
# 필요에 따라 requires_grad 를 False로 설정하여 freeze
for name, param in model.named_parameters:
    print( f"{name} : {param.shape} ({param.requires_grad})" )
    # if name == '???'
    #     param.requires_grad = False

# NMS 적용 소스 코드

In [ ]:
import time

### x, y, w, h => x1, y1, x2, y2 변환 ###
def xywh2xyxy(x):
    y = x.new(x.shape)
    y[..., 0] = x[..., 0] - x[..., 2] / 2
    y[..., 1] = x[..., 1] - x[..., 3] / 2
    y[..., 2] = x[..., 0] + x[..., 2] / 2
    y[..., 3] = x[..., 1] + x[..., 3] / 2
    return y

### Non-maximum suppression 구현 코드 (Github 참조) ###
def non_max_suppression(prediction, conf_thres=0.25, iou_thres=0.45, classes=None):
    """Performs Non-Maximum Suppression (NMS) on inference results
    Returns:
         detections with shape: nx6 (x1, y1, x2, y2, conf, cls)
    """

    nc = prediction.shape[2] - 5  # number of classes

    # Settings
    # (pixels) minimum and maximum box width and height
    max_wh = 4096
    max_det = 300  # maximum number of detections per image
    max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
    time_limit = 1.0  # seconds to quit after
    multi_label = nc > 1  # multiple labels per box (adds 0.5ms/img)

    t = time.time()
    output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]

    for xi, x in enumerate(prediction):  # image index, image inference
        # Apply constraints
        # x[((x[..., 2:4] < min_wh) | (x[..., 2:4] > max_wh)).any(1), 4] = 0  # width-height
        x = x[x[..., 4] > conf_thres]  # confidence

        # If none remain process next image
        if not x.shape[0]:
            continue

        # Compute conf
        x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf

        # Box (center x, center y, width, height) to (x1, y1, x2, y2)
        box = xywh2xyxy(x[:, :4])

        # Detections matrix nx6 (xyxy, conf, cls)
        if multi_label:
            i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
            x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
        else:  # best class only
            conf, j = x[:, 5:].max(1, keepdim=True)
            x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]

        # Filter by class
        if classes is not None:
            x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

        # Check shape
        n = x.shape[0]  # number of boxes
        if not n:  # no boxes
            continue
        elif n > max_nms:  # excess boxes
            # sort by confidence
            x = x[x[:, 4].argsort(descending=True)[:max_nms]]

        # Batched NMS
        c = x[:, 5:6] * max_wh  # classes
        # boxes (offset by class), scores
        boxes, scores = x[:, :4] + c, x[:, 4]
        i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
        if i.shape[0] > max_det:  # limit detections
            i = i[:max_det]

        output[xi] = x[i]

        if (time.time() - t) > time_limit:
            print(f'WARNING: NMS time limit {time_limit}s exceeded')
            break  # time limit exceeded

    return output

In [ ]:
input_img = (img.transpose(2,0,1)).astype(np.float32) / 255. # normalize & transpose
input_img = torch.from_numpy( input_img ).unsqueeze(0)

if torch.cuda.is_available():
    input_img = input_img.to("cuda")

# Get detections
with torch.no_grad():
    detections = model(input_img)

In [ ]:
nms_result = non_max_suppression( detections, conf_thres=0.5, iou_thres=0.5 )[0].to('cpu').numpy()

In [ ]:
nms_result.shape

In [ ]:
# 결과 시각화
plot_image = img.copy()

for (x1, y1, x2, y2, confidence, cls) in nms_result:

    class_name = classes[ int(cls) ]

    b, g, r = cls_color[class_name]

    text = f'{class_name} : {confidence:.2f}'
    size, baseline = cv2.getTextSize( text, 4, 0.9, 2 )
    cv2.putText( plot_image, text, (int(x2-size[0]), int(y2+size[1])), 4, 0.9, (b, g, r), 2 )

    cv2.rectangle( plot_image, (int(x1), int(y1)), (int(x2), int(y2)), (b, g, r), 2 )

plt.figure( figsize=(10,10) )
plt.imshow( plot_image[..., ::-1] )
plt.show()